# Vorbereitung eines Korpus für [TRACER](https://www.etrap.eu/research/tracer/)

## Laden des Textes
Zu tun: Pfad zur Datei und Variable (name) sowie Zeilennummer (line_cnt) anpassen

In [37]:
name = "Aldama-337-Text-PG"
line_cnt = 4000001
text = open("/home/stockhausen/Sync/Severian/Plaintext-Korpus/%s.txt" % name, encoding='utf-8').read()

## Normalisierung des Textes

In [38]:
from cltk.corpus.utils.formatter import cltk_normalize
normalized = cltk_normalize(text).lower()

## Aufteilung des Textes in Sätze 

In [39]:
from cltk.tokenize.sentence import TokenizeSentence
tokenizer = TokenizeSentence('greek')
tokenized = tokenizer.tokenize_sentences(normalized)

## Korpus als TSV schreiben
Diese Datei wird von TRACER zum Textvergleich verwendet.

In [40]:
with open("%s-Korpus.txt" % name, "w") as korpus:
    for lines in tokenized:
        korpus.write(str(line_cnt)+'\t'+lines+"\t"+"NULL"+"\t"+name+"\n")
        line_cnt+=1

## POS-Tagging und Lemmatisierung
ACHTUNG: CLTK-Lemmatizer und -POSTagger liefern auch falsche Ergebnisse!
TRACER-Format: word-form TAB base-form TAB word-class (POS-Tag 1. Buchstabe)

### POS-Tagging

In [41]:
from cltk.tag.pos import POSTag
tagger = POSTag('greek')
tagged = tagger.tag_crf(normalized)
pos = []
count = 0
for lines in tagged: 
    pos.append(tagged[count][1][0].lower())
    count+=1

### Lemmatisierung

In [42]:
from cltk.stem.lemma import LemmaReplacer
lemmatizer = LemmaReplacer('greek')
lemmatized = lemmatizer.lemmatize(normalized, return_raw=True)

### Lemma-Liste im Format für TRACER

In [43]:
with open("%s.lemma" % name, "w") as lemma:
    master = [list(l) for l in zip(lemmatized, pos)]
    for i, v in enumerate(master):
       if v[0] == ',/,' or v[0] == ';/;':
           master.remove(master[i])
    bereinigt = [[x.replace('/','\t') for x in l] for l in master]
    fertig = '\n'.join('\t'.join(inner) for inner in bereinigt)
    lemma.write(fertig)